# Robot Learning Visualizations

#### Basic Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# Change some settings for better aesthetics
mpl.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    # "axes.labelsize": 15,
    # "axes.titlesize": 16,
    # "legend.fontsize": 12,
    # "xtick.labelsize": 12,
    # "ytick.labelsize": 12,
    # "lines.linewidth": 2.0,
})

# Enable high-definition inline plotting
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # or 'svg' for vector


# Optional: other style
sns.set_theme(style="darkgrid")

# sns.set_theme(
#     style="darkgrid",
#     rc={
#         "font.family": "serif",
#         "mathtext.fontset": "cm",
#         "axes.labelsize": 15,
#         "axes.titlesize": 16,
#         "legend.fontsize": 12,
#         "xtick.labelsize": 12,
#         "ytick.labelsize": 12,
#         "lines.linewidth": 2.5,
#     }
# )

# plt.rcParams['font.family'] = 'serif'
# plt.rcParams['mathtext.fontset'] = 'cm'


#### Load Evaluation Log

In [ ]:
# df = pd.read_csv("eval_logs/eval_log.csv")
#df = pd.read_csv("eval_logs/PPO_10deg_rand_force.csv")
#df = pd.read_csv("eval_logs/PPO_10deg_rand_force_4.csv")
#df = pd.read_csv("eval_logs/PPO_10deg_rand_force_4_0force.csv")
# df = pd.read_csv("eval_logs/PPO_10deg_rand_force_5.csv")
# df = pd.read_csv("eval_logs/PPO_noDistur_fasterCurr_newPen_30deg.csv")
df = pd.read_csv("eval_logs/PPO_noDistur_fasterCurr_newPen_30deg_evalulated_with_force.csv")
# df = pd.read_csv("eval_logs/PPO_noDistur_fasterCurr_newPen_30deg_evalulated_without_force.csv")


print("Data loaded:", df.shape)
df.head()


#### Reward per Step

In [ ]:
df.groupby("step")["reward"].sum().plot(figsize=(10, 3.5))
plt.ylabel("Reward", fontsize=13)
plt.xlabel("Time Step", fontsize=13)
plt.ylim(top=1.006)

plt.axhline(y=1.0, color="black", linestyle="--", linewidth=1)  # Dashed line at y=1

#### Physical Parameters over Time (in one Plot)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(9, 7), sharex=True)

# --- Positions ---
axes[0].plot(df["x"], label=r"$x$ position")
axes[0].plot(df["y"], label=r"$y$ position")
axes[0].plot(df["z"], label=r"$z$ position")
axes[0].set_title(r"Position", fontsize=15)
axes[0].set_ylabel(r"Position  $\mathrm{[m]}$", fontsize=15)
axes[0].legend(loc="center left", bbox_to_anchor=(1.02, 0.5), borderaxespad=0)

# --- Angles ---
axes[1].plot(df["x_angle"], label=r"$\theta_x$ angle")
axes[1].plot(df["y_angle"], label=r"$\theta_y$ angle")
axes[1].plot(df["z_angle"], label=r"$\theta_z$ angle")
axes[1].set_title(r"Angles", fontsize=15)
axes[1].set_ylabel(r"Angle  $\mathrm{[deg]}$", fontsize=15)
axes[1].legend(loc="center left", bbox_to_anchor=(1.02, 0.5), borderaxespad=0)

# --- Velocity ---
axes[2].plot(df["x_vel"], label=r"velocity in"+"\n"+r"$x$ direction")
axes[2].set_title(r"Velocity in $x$ direction", fontsize=15)
axes[2].set_ylabel(r"$\dot{x}$  $[\mathrm{m/s}]$", fontsize=15)
axes[2].legend(loc="center left", bbox_to_anchor=(1.02, 0.5), borderaxespad=0)
axes[2].set_xlabel("Time step", fontsize=14)


# plt.tight_layout()
plt.subplots_adjust(right=0.8)  # leave space for legends on the right
plt.show()
# plt.tight_layout()

### Action over Time

In [ ]:
action_cols = [col for col in df.columns if "action" in col or "wheel_speed" in col]
axes = df[action_cols].plot(subplots=True, figsize=(9, 8))

plt.suptitle(r"Wheel Speed and Action (Torque)", y=0.92, fontsize=15)
labels = {
    "action_0": r"Torque Left" + "\n" + r"$\text{[Nm]}$",
    "action_1": r"Torque Right" + "\n" + r"$\text{[Nm]}$",
    "wheel_speed_l": r"$\dot\theta_\text{left wheel}$" + "\n" + r"$[\text{deg/s}]$",
    "wheel_speed_r": r"$\dot\theta_\text{right wheel}$" + "\n" + r"$[\text{deg/s}]$",
}
for ax, col in zip(axes, action_cols):
    ax.set_ylabel(labels.get(col, col), fontsize=15)  # Fallback to column name if label missing
    # ax.legend(loc="upper right")
axes[-1].set_xlabel("Time step", fontsize=14);

### Individual Penalties over Time

#### First, just the bounded sensor data *without* penalty factors $\lambda_{i}$

The bounded sensor data lies between $[0, 1]$ (for plotting here in $[-1, 0]$ actually)

In [ ]:
info_cols = [col for col in df.columns if "bounded" in col]
axes = df[info_cols].plot(subplots=True, figsize=(10, 12))

plt.suptitle(r"Bounded Sensor Data Observations $f_{s_i}(o_i)$ without Penalty Factors $\lambda_i$", y=0.91)
labels = {
    "bounded_dist": r"$f_{s_{x}}(x)$",
    "bounded_y_angle": r"$f_{s_{\theta_y}}(\theta_y)$",
    "bounded_wheel_l": r"$f_{s_{\dot \theta_\text{L wheel}}}\left( \dot \theta_\text{L wheel}\right)$",
    "bounded_wheel_r": r"$f_{s_{\dot \theta_\text{R wheel}}}\left( \dot \theta_\text{R wheel}\right)$",
    "bounded_z_angle": r"$f_{s_{\theta_z}}(\theta_z)$",
    "bounded_x_vel": r"$f_{s_{\dot x}}(\dot x)$",
    "bounded_y_angle_vel": r"$f_{s_{\dot \theta_y}}(\dot \theta_y)$",
}
for ax, col in zip(axes, info_cols):
    ax.set_ylabel(labels.get(col, col), fontsize=13)  # Fallback to column name if label missing
    ax.legend(loc="lower right")

for ax in axes:
    ax.set_ylim(-1.025, 0.025)  # Adjust limits as needed
axes[-1].set_xlabel("Time step");

#### Now, *with* penalty factors $\lambda_{i}$

Now, the penalties are scaled with by their respective penalty factors $\lambda_{i}$ (but still between $[0, 1]$ or $[-1, 0]$ for plotting)
This shows each individual penalty influences the final reward.

In [ ]:
info_cols = [col for col in df.columns if "penalty" in col or "info_" in col]
axes = df[info_cols].plot(subplots=True, figsize=(10, 14))

plt.suptitle(r"Full Penalties of Observations $\lambda_{i} f_{s_i}(o_i)$", y=0.91)
labels = {
    "distance_penalty": r"$\lambda_x f_{s_x}(x)$",
    "y_angle_penalty": r"$\lambda_{\theta_y} f_{s_{\theta_y}}(\theta_y)$",
    "z_angle_penalty": r"$\lambda_{\theta_z} f_{s_{\theta_z}}(\theta_z)$",
    "wheel_l_penalty": r"$\lambda_{\dot \theta_\text{wheel}} f_{s_{\dot \theta_\text{L wheel}}}\left( \dot \theta_\text{L wheel}\right)$",
    "wheel_r_penalty": r"$\lambda_{\dot \theta_\text{wheel}} f_{s_{\dot \theta_\text{R wheel}}}\left( \dot \theta_\text{R wheel}\right)$",
    "x_vel_penalty": r"$\lambda_{\dot x} f_{s_{\dot x}}(\dot x)$",
    "y_angle_vel_penalty": r"$\lambda_{\dot \theta_y} f_{s_{\dot \theta_y}}(\dot \theta_y)$",
}
for ax, col in zip(axes, info_cols):
    ax.set_ylabel(labels.get(col, col), fontsize=13)  # Fallback to column name if label missing
    ax.legend(loc="lower right")

for ax in axes:
    ax.set_ylim(-0.5, 0.025)  # Adjust limits as needed

axes[-1].set_xlabel("Time step");
